# Author : Alex J. Deleon
# Last Modified : October 1, 2020

In [2]:
import numpy as np
import requests
import os
import pandas as pd
from itertools import groupby
from operator import itemgetter
from functools import reduce
import timeit
from pandas.io.json import json_normalize
import json

### DO NOT EDIT CODE BLOCK

In [3]:
pr_counties = ["001","003","005","007","009","011",
"013","015","017","019","021","023","025","027","029","031","033","035","037","039","041","043","045","047",
"049","051","053","054","055","057","059","061","063","065","067","069","071","073","075","077","079","081",
"083","085","087","089","091","093","095","097","099","101","103","105","107","109","111","113","115","117",
"119","121","123","125","127","129","131","133","135","137","139","141","143","145","147","149","151","153"]

In [4]:
def seperateVariable(dfs, blocks = bool, filename = str):
    geoid = {'state':'','county':'','county subdivision':'','tract':'','block group':''}
    dfs = dfs if isinstance(dfs,list) else [dfs]
    for df in dfs:
        for col in df.columns:
            if col in geoid:   
                geoid[col] = col    
    print (dfs)         
    geoidContent = list(filter(None,geoid.values()))
    print (geoidContent)
    if blocks:
        final_df = pd.concat(dfs)
        print (final_df)
    else:
        if not isinstance(dfs,list):
            dfs = [dfs]
        if len(dfs) ==1:
            final_df = dfs[0]
        else:
            print ()
            final_df = reduce(lambda left,right: pd.merge(left,right,on=['NAME'] + geoidContent), dfs)
    print (final_df)

    final_df['GEOID'] = final_df[geoidContent].agg(''.join, axis=1)
    for col in final_df.columns:
        for variableName, variable in variables.items():
            if str(col) == variable:
                final_df.rename(columns={str(col): variableName}, inplace=True)    
                
    print (final_df)
    
    path = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop','CENSUS_DATA')
    if not os.path.exists(path):
        os.mkdir(path)
    outPath = os.path.join(path,filename)
    final_df.to_csv(outPath + ".csv", index=False, header=True,encoding='iso-8859-1')
    
baseurl = "https://api.census.gov/data/"

class parseJson:
    def __init__(self,url, dataset, year, endpt=None):
        if endpt:
            if type(endpt) is dict:
                endpt_list = list(endpt.values())
            elif type(endpt) is list:
                endpt_list = endpt 
            else:
                endpt_list = list(endpt)

            parse_dict = {letter:list(endpts)  for letter, endpts in groupby(sorted(endpt_list), key=itemgetter(0))}
            addComma = lambda x: ",".join(x)
           # print (parse_dict)
            if "S" in parse_dict:
                subject =  addComma(parse_dict["S"]) #pulls endpts from subject table using "S" key
                self.subject = subject

            if "B" in parse_dict:
                detailed = addComma(parse_dict["B"]) #pulls endpts from detailed table using "B" key
                self.detailed = detailed

            if "D" in parse_dict:
                data = addComma(parse_dict["D"])     #pulls endpts from data table using "D" key
                self.data = data

            if "P" in parse_dict:
                dec_p = addComma(parse_dict["P"])
                self.dec = dec_p


            if "H" in parse_dict:
                dec_h = addComma(parse_dict["H"])

                if hasattr(self,'dec'):
                    print ('yes')
                    self.dec += ',' + dec_h
                else:
                    self.dec = dec_h
               
        self.url = url + str(year) + "/" + dataset + "/"
        print (self.url)
   
    def _queryHelper(self,state = None,county=None):
        query_county = "&in=county:{}".format(county) if county else ""
        query_county_state = "&in=state:{}".format(state)+query_county if state else " "
        return query_county_state
    
    def getData(self, geo = None,state = None,county=None):
        self.data_url = self.url+"profile?get=NAME,COUNTY,"+self.data+"&for="+geo+":*"+self._queryHelper(state,county)
        print (self.data_url)
        return self.convertToDF(self.pullData(self.data_url))
   
    def getDetail(self, geo = None,state = None,county=None):
        self.detail_url = self.url+"?get=NAME,COUNTY,"+self.detailed+"&for="+geo+":*"+self._queryHelper(state,county)
        print (self.detail_url)
        return self.convertToDF(self.pullData(self.detail_url))
         
    def getSubject(self,geo = None, state = None,county=None):
        self.subject_url = self.url+"subject?get=NAME,COUNTY,"+self.subject+"&for="+geo+":*"+self._queryHelper(state,county)
        print (self.subject_url)
        return self.convertToDF(self.pullData(self.subject_url))
    
    def getcbp(self,geo = None, state = None,county=None):
        self.cdp_url = self.url+"?get=ESTAB,GEO_ID,LFO,LFO_LABEL,EMPSZES_LABEL,EMPSZES,EMP&for="+geo+":*"+self._queryHelper(state,county)
        print (self.cdp_url)
        return self.convertToDF(self.pullData(self.cdp_url))
        
    def getDecennial(self,geo=None,state = None,county=None):
        self.decennial_url = self.url+"?get=NAME,"+self.dec+"&for="+geo+":*"+self._queryHelper(state,county)
        print (self.decennial_url)
        return self.convertToDF(self.pullData(self.decennial_url))
        
    def getDetail_Blocks(self):
        dfs = []
        for county in pr_counties:
            self.detail_url_blocks = self.url + "?get=NAME," + self.detailed + "&for=block%20group:*&in=state:72%20county:" + county
            print ( self.detail_url_blocks )
            dfs.append(self.convertToDF(self.pullData(self.detail_url_blocks)))
        return dfs
    
    def getSubject_Blocks(self):
        dfs = []
        for county in pr_counties:
            self.subject_url_blocks = self.url + "subject?get=NAME," + self.subject + "&for=block%20group:*&in=state:72%20county:" + county
            dfs.append(self.convertToDF(self.pullData(self.subject_url_blocks)))
        return dfs    
    
    def getData_Blocks(self):
        dfs = []
        for county in pr_counties:
            self.data_url_blocks = self.url + "profile?get=NAME," + self.data + "&for=block%20group:*&in=state:72%20county:" + county
            dfs.append(self.convertToDF(self.pullData(self.data_url_blocks)))
        return dfs    
    
    def pullData(self, geturl):
        req = requests.get(geturl)
        data = req.json()
        print (data)
        return data
    
    def convertToDF(self,json):
        indices = [i[0].split(',')[0] for i in json[1:]]
        df = pd.DataFrame(json[0:][1:], indices, json[0][0:])
        return df
    
    def getAll(self, geo = None, state = None,county=None,filename = str):
        tables = {key:value for key, value in self.__dict__.items()}
        print (tables)
        methods = {'subject': self.getSubject, 'detailed':self.getDetail,'data':self.getData, 'dec': self.getDecennial}
        dfs = []
        for attr in tables:
            if attr != 'url':
            #    print (attr)
                dfs.append(methods[attr](geo = geo,state= state,county=county))
        #print (dfs)
        seperateVariable(dfs,blocks = False,filename = filename)
        
    def getAll_blocks(self, filename = str):
        tables = {key:value for key, value in self.__dict__.items()}
        methods = {'subject': self.getSubject_Blocks, 
                   'detailed':self.getDetail_Blocks,
                   'data':self.getData_Blocks}
        dfs_list = []
        for attr in tables:
            if attr != 'dec' and  attr != 'url':
                dfs_list.append(methods[attr]())
        dfs = [df for df_list in dfs_list for df in df_list]
        #print (dfs)
        seperateVariable(dfs,blocks = True,filename = filename)

#  Insert variables here. Please make sure variables are in this form:

    {name 1: variable 1,
     name 2: variable 2,
        ...
     name k: variable k}

### Example: 

    {"Total": "B25002_001E",
    "Occupied":"B25002_002E",
    "Vacant" : "B25002_003E"}

### NOTES: 

* Make sure the name and variable are wrapped wit quotes "".
* Double check if variables are correctly typed

In [1]:
variables = {  
#                'WHITE_POP':'DP05_0077E',
#                'BLACK_POP':'DP05_0078E',
#                'WHITE_POP_PER':'DP05_0037PE',
#                'PR_POP':'DP05_0073E',
#                'HISP_NPR_POP':'DP05_0071E',
#                'MALE':'DP05_0002PE'
    
              'TOTPOP':'S0101_C01_001E',
#               'MEXPOP':'B03001_004E',
#               'CUBANPOP':'B03001_006E',
#               'DOMPOP':'B03001_007E',
#               'CENTRALPOP':'B03001_008E',
#               'SOUTHPOP':'B03001_016E',
#               'OTHER_HISP':'B03001_027E',
    
#               'TOTPOP':'B02001_001E',
#               'WHITE':'B02001_002E',
#               'BLACK':'B02001_003E',
#               'ASIAN':'B02001_005E',
#               'AMERINDIAN':'B02001_004E',
#               'NATIVE_HAW':'B02001_006E',
#               'OTHER':'B02001_007E'
#               'TOTEMP':'S2401_C01_001E',
#               'TOTHH':'S2501_C01_001E',
#              'PTM':'S2401_C01_033E',
#              'NCM':'S2401_C01_029E',
#              'SERVICE':'S2401_C01_018E',
#              "DISABLED": "DP02_0071PE",
#              'HEALTH':'S2401_C01_015E',
#              'PUBTRANS':'S0801_C01_009E',
#              '4CARPOOL':'S0801_C01_007E',
#              'NOVEH':'DP04_0058PE',
#              'GROUPQ':'B26001_001E',
#              'OVER65':'DP02_0076PE',
 #             '25NOHSDIP':'DP02PR_0062PE',
#              'LABORFORCE':'S2301_C02_001E',
#                'BELOWPOV':'DP02PR_0059PE',
#                 'VACANT':'DP04_0003PE',
#    
#                "UNEMP3":"S2301_C04_004E",
#                "UNEMP4":"S2301_C04_005E",
#                "UNEMP5":"S2301_C04_006E",
#                "UNEMP6":"S2301_C04_007E",
                 "EP_POV":"S0601_C01_049E",
#                "MED_INC":"S1901_C01_012E",
#                 'UNEMP':'DP03_0005PE',
#                 'MEDINC':'DP03_0062E',
                
#                'F_SING':'DP02_0009PE',
#                'M_SING':'DP02_0007PE',
#                'PROT_SERV':'S2401_C01_020E',
#                'HTH_SERV':'S2401_C01_019E'
#              'RENT35PER':'DP04_0142PE',
#              'NOCOMP':'S2801_C01_011E',
#               'NOINTERNET':'S2801_C01_019E',
#               'DIS_NOINSUR':'B18135_007E',
#                'NOINSUR':'DP03_0099PE',
#                'AVG_HH':'DP02_0015E',
#               'RM1_150':'S2501_C01_007E',
#               'RM150_UP':'S2501_C01_008E',
#                 'ASIAN':'DP05_0080PE',
#                 'MEXICAN':'DP05_0072PE',
#                 'BLACK':'DP05_0078PE',
#                 'WHITE':'DP05_0077PE',
#                 'HISP':'DP05_0071PE',
#                 'PR':'DP05_0073PE',
#                 'GRAND':'DP02_0044PE',
    
    #'UNWEIGHTED_POP':'B00001_001E',
     #        'WEIGHTED_POP':'B01001_001E'
#              "TOTAL_POP_18":"P005001",
#              "TOTAL_POP":"P001001",
#               "ONERACE":"P005002",
#               "WHITE":"P005003",
#               "BLACK":"P005004",
#               "AMERINDIAN_ALSK":"P005005",
#               "ASIAN":"P005006",
#               "NATHAW_PACIS":"P005007",
#               "OTHER":"P005008"
    
#"BirthsUnmarried":'S1301_C05_001E',
            # "WomenNeverMarried":'B12002_096E',
            # "WomenNeverMarried_35_39":"B12002_102E",
             #"WomenNeverMarried_40_44":"B12002_103E",
             #"Pop18Over": "S0101_C01_025E"
             
#              "totPop": "P004001",
#              "blackPop": "P005004",
#              "totPopSexAge":"P012001",
#              "mUnder5": "P012003",
#              "fUnder5": "P012027",
#              "m65to66":"P012020",
#              "m67to69":"P012021",
#              "m70to74":"P012022",
#              "m75to79":"P012023",
#              "m80to84":"P012024",
#              "m85Over":"P012025",

#              "f65to66":"P012044",
#              "f67to69":"P012045",
#              "f70to74":"P012046",
#              "f75to79":"P012047",
#              "f80to84":"P012048",
#              "f85Over":"P012049",
            
#              "totHH": "H004001",
#              "rentHH": "H004004",

#              "totHHAge": "H018001",
#              "femaleHH": "H018027",
#              "MobHomes": "DP04_0014PE",

#              "femaleEmp": "S2301_C03_021E",
#              "totEmpSub": "S2301_C03_001E",
#              "noFuelHU" : "S2504_C01_037E",
#              "plumbing" : "S2504_C01_024E",
#              "TotOcc" : "S2504_C01_001E",
#              "Pop16Over": "S2301_C01_001E",
#              "UnempRateWithDisability": "S2301_C04_024E",
#              "AvgPerHH": "H012001",
#              "HispPop": "P004003",
             # "NoHSOver25": "S0601PR_C02_033E",
#              "TotOver25" : " S0102_C01_033E",
#              "PopBelowPov" : "B06012PR_002E",
#              "MedValOHH" : "S2506_C01_009E",
#              "MedRent" : "B25071_001E",
#              "ActiveDocs" : "B24126_166E",
#              "MedPrac" : "S2401_C01_016E",
#              "TeleServicePE": "S2504_C01_045E",
#              "TeleService":"S2504_C01_030E",
#              "PerDisabled_CI": "DP02PR_0071PE",
    
#               "EngProf": "S1602_C04_001E",
               "NoInsure" : "S2701_C05_001E",
#               "Age": "S0101_C02_030E",
#               "HouseInc": "S1903_C01_001E",
#               "MedProfCap": "S2401_C01_016E",
#                "DISABLED":"S1810_C03_001E",
#                "UNEMP":"S2301_C04_001E",
#                "UNEMP":"S2301_C04_001E",
#               "NOHSDP":"S1501_C02_009E",
#               "TotOCC": "DP04_0046PE",
#               "MobHomes": "DP04_0014PE",     
#               'TOTPOP':'S0101_C01_001E',    
#               "NoVehicle": "B08201_002E",
#               "TotHHVehicle": "B08201_001E",        
#               "Gini": "B19083_001E",
#               "FamNoWife": "B09005_004E",
#               "FamNoHusband": "B09005_005E",
#               "TotFams" : "B09005_001E",    
#               "RentPropCap": "B25004_003E", 
#                'TOTEMP':'S2401_C01_001E',   
#               "VacantTot" : "B25004_001E"
#                "MEXICAN":"B03001_004E",
#                "CUBAN":"B03001_006E",
#                "DOMIREP":"B03001_007E",
#                "CENTRAL":"B03001_008E",
#                "SOUTH":"B03001_016E",
#                "HISPANIC":"B03001_003E"
              #  "TOTAL":"P003001",
            #    "WHITE":"P003003",
               # "BLACK":"P003004"
            }

In [10]:
variables={
    #"AMBUlATORY":"B18105_001E",
           "BELOW_POV":"B06012PR_002E",
           "65_OVER":"B08101_008E",
           "TOTPOP":"B01001_001E",
           #"WHITE":"B01001A_001E",
          # "NOHS_OVER25_9th":"S1501_C01_007E",
           #"NOHS_OVER25_9_12th":"S1501_C01_008E",
          # "TotOcc":"DP04_0046E",
           "NoInsure" : "S2701_C05_001E"}

In [7]:
#infrastrasture
variables={"TOT_HH":"S2801_C01_001E",
           "EP_COMPUTER":"S2801_C01_003E",
           "EP_BROADBAND":"S2801_C01_014E",
           "NO_VEHICLE":"DP04_0058PE",
           "EP_INTERNET":"S2801_C01_012E"}

In [49]:
#education
variables={"3Over":"S1401_C01_001E",
           "k12_PubSchool":"S1401_C03_003E",
           "k12_PubSchool_Per":"S1401_C04_003E",
           "Age25Up":"S1501_C01_006E",
           "Age25Up_NoHiDiploma":"S0102_C01_034E",
           "Age3_4_Preschool":"S1401_C01_002E",
           "Age3_4":"S1401_C01_013E",
           "Age18_24_PostSec":"S1401_C01_030E",
           "Age18_24":"S1401_C01_029E",
           "TOT_HH":"S2801_C01_001E",
          }
           

In [ ]:
            'POP_25_29' : 'S0101_C01_007E',
            'POP_30_34' : 'S0101_C01_008E',
            'POP_35_39' : 'S0101_C01_009E',
            'POP_40_44' : 'S0101_C01_010E',
            'POP_45_49' : 'S0101_C01_011E',
            'POP_50_54' : 'S0101_C01_012E',
            'POP_55_59' : 'S0101_C01_013E',
            'POP_60_64' : 'S0101_C01_014E',
            'POP_65_69' : 'S0101_C01_015E',
            'POP_70_74' : 'S0101_C01_016E',
            'POP_75_79' : 'S0101_C01_017E',
            'POP_80_84' : 'S0101_C01_018E',
            'POP_85_UP' : 'S0101_C01_019E',

In [7]:
#education take 2
  
variables={   'TOT_POP' : 'S0101_C01_001E',
              'POP_3_4' : 'S1401_C01_013E',
              'POP_5_9' : 'S1401_C01_015E',
            'POP_10_14' : 'S1401_C01_017E',
            'POP_15_17' : 'S1401_C01_019E',
            'POP_18_19' : 'S1401_C01_021E',
            'POP_20_24' : 'S1401_C01_023E',
    

             'ENR_3_4':  'S1401_C01_014E',
             'ENR_5_9':  'S1401_C01_016E',
             'ENR_10_14':'S1401_C01_018E',
             'ENR_15_17':'S1401_C01_020E',
             'ENR_18_19':'S1401_C01_022E',
             'ENR_20_24':'S1401_C01_024E',
   
             'POP_25UP':"S1501_C01_006E",
             'POP_25UP_Less9th':"S1501_C01_007E",
             'POP_25UP_9to12th':"S1501_C01_008E",
           
              #used to remove null barrios
             'TOT_HH':'S2801_C01_001E'
    
          }


In [11]:
variables={'Total':'B25074_001E',
           'RNT30_35_10k'   :'B25074_006E',
           'RNT35_40_10k'   :'B25074_007E',
           'RNT40_50_10k'   :'B25074_008E',
           'RNT50up_10k'    :'B25074_009E',
           
           'RNT30_35_10_19k':'B25074_015E',
           'RNT35_40_10_19k':'B25074_016E',
           'RNT40_50_10_19k':'B25074_017E',
           'RNT50up_10_19k' :'B25074_018E',
                      
           'RNT30_35_20_35k':'B25074_024E',
           'RNT35_40_20_35k':'B25074_025E',
           'RNT40_50_20_35k':'B25074_026E',
           'RNT50up_20_35k' :'B25074_027E',
                      
           'RNT30_35_35_50k':'B25074_033E',
           'RNT35_40_35_50k':'B25074_034E',
           'RNT40_50_35_50k':'B25074_035E',
           'RNT50up_35_50k' :'B25074_036E',
                      
           'RNT30_35_50_75k':'B25074_042E',
           'RNT35_40_50_75k':'B25074_043E',
           'RNT40_50_50_75k':'B25074_044E',
           'RNT50up_50_75k' :'B25074_045E',
        
           'RNT30_35_75_100k'  :'B25074_051E',
           'RNT35_40_75_100k'  :'B25074_052E',
           'RNT40_50_75_100k'  :'B25074_053E',
           'RNT50up_75_100k'   :'B25074_054E',
           
           'RNT30_35_100k'  :'B25074_060E',
           'RNT35_40_100k'  :'B25074_061E',
           'RNT40_50_100k'  :'B25074_062E',
           'RNT50up_100k'   :'B25074_063E',
           
          }

In [13]:
#Health
variables={'NO_HLTH_INSUR':'DP03_0099PE',
           'WITH_DISABL':'DP02PR_0072PE',
           'WITH_DISABL_OTHER':'S1810_C03_001E'
           
          }
           

In [2]:
#housing
variables={#one-var below to get overcrowding
#           "OneOrLessPerRoom":"DP04_0077PE",
#           "LACK_PLUMBING":"DP04_0073PE",
#             "LACK_KITCHEN":"DP04_0074PE",
    
#            #constructed var
#             'Built_2014_later':'DP04_0017PE',
#             'Built_2010_2013':'DP04_0018PE',
#             'Built_2000_2009':'DP04_0019PE',
#             'Built_1990_1999':'DP04_0020PE',
#             'Built_1980_1989':'DP04_0021PE',
#             'Built_1970_1979':'DP04_0022PE',
#             'Built_1960_1969':'DP04_0023PE',
#             'Built_1950_1959':'DP04_0024PE',
#             'Built_1940_1949':'DP04_0025PE',
#             'Built_1939_earlier':'DP04_0026PE',

#            #rent burden (add below/tothu)
#            "RENT30_35":"DP04_0141E",
#            "RENT35UP":"DP04_0142E",
#            "MORTG30_35":"DP04_0114E",
#            "MORTG35UP":"DP04_0115E",
           
#            #vacancy
#           "VACANCY_RATE":"DP04_0003PE",
    
            #general HH vars
           'TOT_HU':"DP04_0001E",
           'TOT_HH':'S2801_C01_001E'
           }

In [2]:
variables={'WITH_DISABL':'DP02PR_0072E',
            "BELOW_POV":"B06012PR_002E",

            'TOT_POP' : 'S0101_C01_001E',
            'POP_3_4' : 'S1401_C01_013E',
            'POP_5_9' : 'S1401_C01_015E',
            'POP_10_14' : 'S1401_C01_017E',
            'POP_15_17' : 'S1401_C01_019E',
           
            'POP_65_69' : 'S0101_C01_015E',
            'POP_70_74' : 'S0101_C01_016E',
            'POP_75_79' : 'S0101_C01_017E',
            'POP_80_84' : 'S0101_C01_018E',
            'POP_85_UP' : 'S0101_C01_019E'
           }

<h1><center>Exporting ACS Data</center></h1>


## Step 1: In the 'parseJson' class, do the following:

   * Select your dataset of interest: "acs/acs5, acs/acs3,...,dec/sf1,etc." 
   * Next, select the publication year. For acs, you may select years as early as 2009. 
   
###  See below for an example. Here we selected 2012 ACS 5-year data set using  "acs/acs5" in red and 2018 in green:

<h4><center>parseJson(baseurl, <span style="color: #ff0000"> "acs/acs5"</span>,<span style="color: #228B22"> 2012</span>, variables)</center></h4>


## Step 2: In the 'getAll' method, do the following:
   * Set "df" to True (df = True) if you want to export the data selected to your desktop.
   * Select geography of interests, either "Tract" or "County".
   * Select the name you want your output csv to be. (filename = 'someName')
   
### See below for another example. Here we selected df = True, county as our geography, and 'Putnam_SC2' as our filename.
   <h4><center>.getAll(df = <span style="color:#228B22"> True</span>, geo = <span style="color: #ff0000">'County'</span>, filename = <span style="color: #ff0000"> 'Putnams_SC2'</span>)</center></h4>
   
### To complete our example, the full line of code should look like 

 <h4><center>parseJson(baseurl, <span style="color: #ff0000"> "acs/acs5"</span>,<span style="color: #228B22"> 2012</span>, variables).getAll(df = <span style="color:#228B22"> True</span>, geo = <span style="color: #ff0000">'County'</span>, filename = <span style="color: #ff0000"> 'Putnams_SC2'</span>)</center></h4>

In [9]:
# years = [2015,2016,2017,2018]
# for year in years:
#     parseJson(baseurl,"acs/acs5",year,variables).getAll(df = True, geo = 'county',filename = 'VacantHousingAndPop_{}'.format(year))
# #dec = parseJson(baseurl,"dec/sf1",2000,variables).getDecennial(df = True, geo = 'Tract')

In [13]:
variables={'TOT5':'B16004_001E',
'TOT5_17':'B16004_002E',
'T517ENG':'B16004_003E',
'T517S':'B16004_004E',
'T517SEVW':'B16004_005E',
'T517SEW':'B16004_006E',
'T517SENW':'B16004_007E',
'T517SNE':'B16004_008E',
'T517I':'B16004_009E',
'T517IEVW':'B16004_010E',
'T517IEW':'B16004_011E',
'T517IENW':'B16004_012E',
'T517INE':'B16004_013E',
'T517A':'B16004_014E',
'T517AEVW':'B16004_015E',
'T517AEW':'B16004_016E',
'T517AENW':'B16004_017E',
'T517ANE':'B16004_018E',
'T517O':'B16004_019E',
'T517OEVW':'B16004_020E',
'T517OEW':'B16004_021E',
'T517OENW':'B16004_022E',
'T517ONE':'B16004_023E',
'TOT18_64':'B16004_024E',
'T1864ENG':'B16004_025E',
'T1864S':'B16004_026E',
'T1864SEVW':'B16004_027E',
'T1864SEW':'B16004_028E',
'T1864SENW':'B16004_029E',
'T1864SNE':'B16004_030E',
'T1864I':'B16004_031E',
'T1864IEVW':'B16004_032E',
'T1864IEW':'B16004_033E',
'T1864IENW':'B16004_034E',
'T1864INE':'B16004_035E',
'T1864A':'B16004_036E',
'T1864AEVW':'B16004_037E',
'T1864AEW':'B16004_038E',
'T1864AENW':'B16004_039E',
'T1864ANE':'B16004_040E',
'T1864O':'B16004_041E',
'T1864OEVW':'B16004_042E',
'T1864OEW':'B16004_043E',
'T1864OENW':'B16004_044E',
'T1864ONE':'B16004_045E',
'TOT65':'B16004_046E',
'T65ENG':'B16004_047E'}

In [30]:
variables={'TOTHH':'DP04_0001E',
'TOTHH_OCC':'DP04_0002E',
'TOTHH_VAC':'DP04_0003E',
'MED_ROOMS':'DP04_0037E',
'OWN_OCC':'DP04_0046E',
'RENT_OCC':'DP04_0047E',
'AVG_OWN':'DP04_0048E',
'AVG_RENT':'DP04_0049E',
'HH_50K':'DP04_0081E',
'HH_50_99K':'DP04_0082E',
'HH_100_149K':'DP04_0083E',
'HH_150_199K':'DP04_0084E',
'HH_200_299K':'DP04_0085E',
'HH_300_499K':'DP04_0086E',
'HH_500_999K':'DP04_0087E'}




In [5]:
variables={'BUILT_2014_After':'S2504_C01_009E',
           #'BUILT_2010_2013':'S2504_C01_010E',
           #'BUILT_2000_2009':'S2504_C01_011E',
           #'BUILT_1980_1999':'S2504_C01_012E',
           #'BUILT_1960_1979':'S2504_C01_013E',
           #'BUILT_1940_1959':'S2504_C01_014E',
           #'BUILT_1939_Before':'S2504_C01_015E',
          }

## Follow the similiar steps for the 'getAll_blocks' method. See example below. Run it and you should get a csv called 'test1.csv' loaded to your desktop,

In [7]:
# Changable parameters: 
parseJson(baseurl,"acs/acs5",2017,variables).getAll(geo="county%20subdivision",state="72",filename="YRBLT_AFTER2014_ACS2017")

https://api.census.gov/data/2017/acs/acs5/
{'subject': 'S2504_C01_009E', 'url': 'https://api.census.gov/data/2017/acs/acs5/'}
https://api.census.gov/data/2017/acs/acs5/subject?get=NAME,COUNTY,S2504_C01_009E&for=county%20subdivision:*&in=state:72
[['NAME', 'COUNTY', 'S2504_C01_009E', 'state', 'county', 'county subdivision'], ['Magos barrio, San Sebastián Municipio, Puerto Rico', '131', '0', '72', '131', '48776'], ['Piñas barrio, Toa Alta Municipio, Puerto Rico', '135', '0', '72', '135', '62063'], ['Mucarabones barrio, Toa Alta Municipio, Puerto Rico', '135', '0', '72', '135', '55398'], ['Quebrada Arenas barrio, Toa Alta Municipio, Puerto Rico', '135', '0', '72', '135', '66965'], ['Galateo barrio, Toa Alta Municipio, Puerto Rico', '135', '0', '72', '135', '29856'], ['Voladoras barrio, Moca Municipio, Puerto Rico', '099', '0', '72', '099', '87648'], ['Naranjo barrio, Moca Municipio, Puerto Rico', '099', '0', '72', '099', '56387'], ['Capá barrio, Moca Municipio, Puerto Rico', '099', '0', '

In [10]:
import urllib2
r'https://api.census.gov/data/2019/acs/acs5/?get=NAME,COUNTY,B01001A_001E,B01001_001E,B06012_002E,B08101_008E,B18105_001E&for=tract:*&in=state:36'

ModuleNotFoundError: No module named 'urllib2'